# Market Access in CIV

1. Travel time 

In [35]:
import sys, os
import rasterio

import pandas as pd
import geopandas as gpd
import numpy as np
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../../../GOST_Urban")
import src.UrbanRaster as urban

sys.path.append("../../")
import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc
import infrasap.osm_extractor as osm
from infrasap.misc import tPrint

In [17]:
# Define local data
in_folder = "/home/wb411133/data/Country/CIV/PROJECTS/MarketAccess"
in_admin_file = os.path.join(in_folder, 'iv2_index.shp')
in_cities_file = os.path.join(in_folder, 'CIV city coordinates.csv')

inB = gpd.read_file(in_admin_file)
inC = pd.read_csv(in_cities_file, encoding = "ISO-8859-1")
inC_geom = inC.apply(lambda x: Point(x['Longitude'], x['Latitude']), axis=1)
inC = gpd.GeoDataFrame(inC, geometry=inC_geom, crs={'init':'epsg:4326'})
inC.to_file(in_cities_file.replace(".csv", ".geojson"), driver="GeoJSON")

In [32]:
city_bounds = gpd.GeoDataFrame(pd.DataFrame([[1,box(*inC.total_bounds)], [2, box(*inB.total_bounds)]], columns=['ID','geometry']), geometry='geometry', crs={'init':'epsg:4326'})
city_bounds['geometry'] = city_bounds['geometry'].buffer(0.5)

In [33]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
inG = rasterio.open(global_friction_surface)
inP = rasterio.open(global_population)

# Create local files
local_friction_file = os.path.join(in_folder, "2020_motorized_friction_surface.tif")
local_pop_file = os.path.join(in_folder, "ppp_2020_1km_Aggregated.tif")

rMisc.clipRaster(inG, city_bounds, local_friction_file)
rMisc.clipRaster(inP, city_bounds, local_pop_file)

In [36]:
osm_pbf = os.path.join(in_folder, "ivory-coast-latest.osm.pbf")

# Extract OSM data for airports, ports, and railways
airports = os.path.join(in_folder, "OSM_Airports.geojson")
ports = os.path.join(in_folder, "OSM_Ports.geojson")
rail_stations = os.path.join(in_folder, "OSM_rail_stations.geojson")

pois = osm.load_pois(osm_pbf, city_bounds.unary_union)
pois['airports'].to_file(airports, driver="GeoJSON")
pois['ports'].to_file(ports, driver="GeoJSON")
pois['rail_stations'].to_file(rail_stations, driver="GeoJSON")

In [46]:
# Extract international airports and ports
international_ports = "/home/public/Data/GLOBAL/INFRA/PORTS/attributed_ports.shp"
international_airports = "/home/public/Data/GLOBAL/INFRA/AIRPORTS/International_Airports_Volume.shp"
I_ports = os.path.join(in_folder, "International_Ports.geojson")
A_ports = os.path.join(in_folder, "International_Airports.geojson")

inP = gpd.read_file(international_ports)
selP = inP.loc[inP['geometry'].apply(lambda x: x.intersects(city_bounds.unary_union))]
selP.to_file(I_ports, driver="GeoJSON")

inA = gpd.read_file(international_airports)
selP = inA.loc[inA['geometry'].apply(lambda x: x.intersects(city_bounds.unary_union))]
selP.to_file(A_ports, driver="GeoJSON")

# Run Access Mapping

In [47]:
inG = rasterio.open(local_friction_file)
inG_data = inG.read() * 1000
# Correct no data values
inG_data[inG_data < 0] = 99999999
mcp = graph.MCP_Geometric(inG_data[0,:,:])

In [51]:
# Travel time to cities
out_file = os.path.join(in_folder, "MA_Cities.tif")
if not os.path.exists(out_file):
    ma.calculate_travel_time(inG, mcp, inC, out_file)

# Travel time to international airports
out_file = os.path.join(in_folder, "MA_AllAirports.tif")
if not os.path.exists(out_file):
    inA = gpd.read_file(airports)
    ma.calculate_travel_time(inG, mcp, inA, out_file)
    
# Travel time to international airports
out_file = os.path.join(in_folder, "MA_InternationalAirports.tif")
if not os.path.exists(out_file):
    inA = gpd.read_file(international_airports)
    ma.calculate_travel_time(inG, mcp, inA, out_file)
    
# Travel time to ports
out_file = os.path.join(in_folder, "MA_ports.tif")
if not os.path.exists(out_file):
    inA = gpd.read_file(international_ports)
    ma.calculate_travel_time(inG, mcp, inA, out_file)
    
# Travel time to rail stations
out_file = os.path.join(in_folder, "MA_railStations.tif")
if not os.path.exists(out_file):
    inA = gpd.read_file(rail_stations)
    ma.calculate_travel_time(inG, mcp, inA, out_file)

# Calculate Urban

In [54]:
urban_raster = os.path.join(in_folder, "URBAN.tif")
urban_pop_raster = os.path.join(in_folder, "URBAN_POP.tif")
hd_urban_raster = os.path.join(in_folder, "HD_URBAN.tif")
hd_urban_pop_raster = os.path.join(in_folder, "HD_URBAN_POP.tif")
calc_urban = urban.urbanGriddedPop(local_pop_file)
urban_extents = calc_urban.calculateUrban(densVal=300, totalPopThresh=5000,
                          raster=urban_raster, raster_pop=urban_pop_raster, 
                          print_message='CIV', verbose=True)

hd_urban_extents = calc_urban.calculateUrban(densVal=1500, totalPopThresh=50000,
                          raster=hd_urban_raster, raster_pop=hd_urban_pop_raster, 
                          print_message='CIV', verbose=True, smooth=True, queen=True)

10:25:46	CIV: Read in urban data
10:25:46	CIV: Creating Shape 0
10:25:49	CIV: Creating Shape 1000
10:25:53	CIV: Creating Shape 2000
10:25:54	CIV: Read in urban data
10:25:55	CIV: Creating Shape 0
